In [1]:
# from bs4 import BeautifulSoup
# from selenium import webdriver
# # from selenium.webdriver.firefox.options import Options
# from selenium.webdriver.chrome.options import Options
# import re
# import sys
# import time
# import numpy as np
# import pandas as pd
# import os
# import psycopg2
# from tqdm import tqdm


# def main():

#     docs = np.arange(10000,20000,1)
#     limit = 10

#     def html(url):
#         try:
#             driver.get(url)
#             html = driver.page_source
#             if type(html) == str:
#                 return(html)
#             else:
#                 return('ERROR')
#         except:
#             return('ERROR')

#     def headlessDriver():
#         options = Options()
#         options.add_argument("--headless")
#         options.add_argument('log-level=1')     
#         driver = webdriver.Chrome(options=options, executable_path="chromedriver.exe")
#         return(driver)


#     def buildEntry(index_value,url):
#         t_start = time.time()    
#         index = index_value
#         pageSource = html(url)
#         pageText = str()
#         if pageSource == 'ERROR':
#             pageText = 'ERROR'
#         else:
#             try:
#                 soup=BeautifulSoup(pageSource, 'lxml')
#                 pageText = soup.find(class_='content').text
#                 if pageText == '':
#                     pageText = None
#             except:
#                 pageText = 'ERROR'

#         entry = (pageSource,
#                  pageText,
#                  index)

#         t_end = time.time()
#         duration = t_end - t_start
#         if duration >= 10:
#             time.sleep(np.random.randint(5,15))          
#         return(entry)


#     def fullHarvest(doclist=docs, limit=limit):
#         t = time.time()

#         doclist = tuple(doclist)    
#         limit = 10

#         conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                             database="postgres",
#                             user="tokyo3",
#                             password="evangelion")

#         cur = conn.cursor()

#         command = """
#         SELECT doc_id,doc_url
#         FROM documents
#         WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_skip != 1 and doc_id IN {}
#         LIMIT {}
#         """.format(doclist,limit)

#         cur.execute(command)
#         existing = cur.fetchall()
#         cur.close()
#         conn.close()

#         submission = [buildEntry(int(x[0]),str(x[1])) for x in existing]    
#         skiplist = []
#         newSubmission = []

#         for i in submission:
#             if i[1] == 'ERROR':
#                 skiplist.append(i[2])
#             else:
#                 newSubmission.append(i)

#         sub_len = len(newSubmission)


#         conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                             database="postgres",
#                             user="tokyo3",
#                             password="evangelion")

#         cur = conn.cursor()

#         if len(skiplist) > 1:
#             sql = """
#             UPDATE documents
#             SET doc_skip = 1
#             WHERE doc_id IN {};
#             """.format(tuple(skiplist))
#             cur.execute(sql)
#             conn.commit()
#         if len(skiplist) == 1:
#             sql = """
#             UPDATE documents
#             SET doc_skip = 1
#             WHERE doc_id = {};
#             """.format(skiplist[0])
#             cur.execute(sql)
#             conn.commit()


#         sql = """
#             UPDATE documents
#             SET doc_html = %s,doc_content = %s
#             WHERE doc_id = %s;
#             """

#         cur.executemany(sql,newSubmission)
#         conn.commit()


#         command = """
#         SELECT COUNT(doc_id)
#         FROM documents
#         WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_skip != 1 and doc_id IN {}
#         """.format(doclist)
#         cur.execute(command)
#         count = cur.fetchall()
#         cur.close()
#         conn.close()

#         nt = time.time()
#         dt = nt - t   

#         return(len(skiplist),count,dt,sub_len,existing)


#     counter = 0
#     driver = headlessDriver()

#     for i in tqdm(range(1000)):    
#         s,r,t,l,e = fullHarvest(doclist=docs, limit=limit)
#         print("{} : {:.4f}s : {} skips : {} remaining : {} counter".format(i,t,s,r,counter))
#         if l == 0:
#             print(e)
#             counter += 1
#             if counter > 10:
#                 break
#             else:
#                 driver.quit()
#                 time.sleep(np.random.randint(30,45))
#                 driver = headlessDriver()
                
# if __name__ == '__main__':
#     main()



In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
# from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options
import re
import sys
import time
import numpy as np
import pandas as pd
import os
import psycopg2
from tqdm import tqdm


In [16]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                    database="postgres",
                    user="tokyo3",
                    password="evangelion")

cur = conn.cursor()

command = """
SELECT count(*)
FROM documents
WHERE doc_html LIKE 'ERROR' OR doc_content LIKE 'ERROR'
"""

cur.execute(command)
existing = cur.fetchall()
cur.close()
conn.close()


# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                     database="postgres",
#                     user="tokyo3",
#                     password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT doc_content, COUNT(doc_id)
# FROM documents
# WHERE doc_html IS NOT NULL
# GROUP BY doc_content
# HAVING COUNT(doc_id) > 1
# """

# cur.execute(command)
# existing = cur.fetchall()
# cur.close()
# conn.close()

In [17]:
existing

[(3639,)]